In [1]:
import tensorflow as tf
import keras.backend as K

Using TensorFlow backend.


In [7]:
a = K.constant([[[1, 2], [3, 4]]])
b = K.flatten(a)
c = tf.concat((b, b), axis = 0)

In [8]:
c

<tf.Tensor 'concat_1:0' shape=(8,) dtype=float32>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math
from keras.optimizers import Adam
from datagen import SequenceDataGenerator
from embedding_model import SequenceEmbeddingModel, sequence_loss_with_params
from params import Params
from IPython.display import clear_output
from utils import *
from evaluate import sequence_eval

In [ ]:
EPOCHS = 100
LEARNING_RATE = 1e-4
nan = float('nan')

In [ ]:
params = Params()

params.EMBEDDING_DIM            = 6
params.BATCH_SIZE               = 1
params.NUM_SHAPE                = 3
params.NUM_CLASSES              = params.NUM_SHAPE + 1
params.NUM_FILTER               = [256, 256, 128]
params.ETH_MEAN_SHIFT_THRESHOLD = 1.5
params.DELTA_VAR                = 0.5
params.DELTA_D                  = 1.5
params.IMG_SIZE                 = 256
params.OUTPUT_SIZE              = 64
params.SEQUENCE_LEN             = 20
params.BACKBONE                 = 'xception'
params.TASK                     = 'sequence'
params.COLORS                   = np.random.random((params.NUM_SHAPE+1, 3))

In [ ]:
model = SequenceEmbeddingModel(params)
optim = Adam(lr = LEARNING_RATE)
loss_function = sequence_loss_with_params(params)
model.compile(optim, loss = loss_function)
clear_output()

In [ ]:
dg = SequenceDataGenerator(
    num_shape    = params.NUM_SHAPE, 
    image_size   = params.IMG_SIZE,
    sequence_len = params.SEQUENCE_LEN)
step = 0
loss_history = []

In [ ]:
def fit_xy(model, x, y, loss_history, step):
    history = model.fit(x, y, batch_size = 1, verbose = False)
    if math.isnan(history.history['loss'][-1]):
        return [1, 0]
    loss_history.append(history.history['loss'][-1])
    if step % 100 == 99:
        clear_output()
        visualize_history(loss_history, 'loss')
    step += 1
    update_progress((step % 100) / 100)
    return loss_history, step

In [ ]:
class_num     = params.NUM_CLASSES
embedding_dim = params.EMBEDDING_DIM

exit = False
for epoch in range(EPOCHS):
    if exit:
        break
    for _ in range(100):
        if exit:
            break
        sequence = dg.get_sequence()
        image_info = sequence[0]
        x, y = prep_half_pair_for_model(image_info, params)
        loss_history, step = fit_xy(model, x, y, loss_history, params, dg, step)
        for i in range(1, len(sequence)):
            prev_image_info = sequence[i-1]
            image_info = sequence[i]
            x, y = prep_half_pair_for_model(prev_image_info, params)
            outputs = np.squeeze(model.predict(x))
            emb = outputs[:, :, (class_num):(class_num + embedding_dim)]
            x, y = prep_pair_for_model(image_info, params, prev_image_info, emb)
            a, b = fit_xy(model, x, y, loss_history, params, dg, step)
            if b:
                loss_history, step = a, b
            else:
                exit = True
                break
        sequence = dg.get_sequence()
        sequence_eval(model, sequence, params)
    if not exit:
        model.save('sequence.h5')